In [14]:
# Necessary imports
import nltk
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [3]:
pos_files = glob.glob("data/clean_positive_train_*.csv")
neg_files = glob.glob("data/clean_negative_train_*.csv")

df_pos_list = [pd.read_csv(open(fp, 'r'), encoding='utf-8', engine='c') for fp in pos_files]
df_neg_list = [pd.read_csv(open(fp, 'r'), encoding='utf-8', engine='c') for fp in neg_files]

In [21]:
# Making it a binary classification with either positive or negative score
def posneg(number):
    if number > 0:
        return 1
    else:
        return 0
    
df[0].head()

,text,score,ups,controversiality,parent_text,parent_score,parent_ups,parent_controversiality,fuzz_qr,fuzz_wr,fuzz_partial,fuzz_partial_tksort,fuzz_tksort,fuzz_tkset
0,gt green elixir rip,1,74,0,considering green elixir gave attack speed wou...,1,47,0,39,86,79,68,39,77
1,exact thought process surprised didnt turn,1,74,0,convinced going marisol valles garcia http www...,1,201,0,24,34,38,38,24,24
2,funny scotty beam clothes,0,-9,0,saying cheap open gas station source costs mil...,1,4,0,30,40,44,40,37,37
3,isnt 99 percent entrepreneurs otherwise would ...,1,74,0,bank americathey exist solely funnel money upward,1,87,0,38,38,41,41,38,38
4,gunna downvote comment oh wait still,0,-9,0,going upvote saw yolo congrats though,1,15,0,41,41,42,42,41,41


Now that we have a small part of clean text we can start to learn the texts to our machine

In [20]:
df = [pd.concat([df_pos, df_neg]) for (df_pos, df_neg) in zip(df_pos_list, df_neg_list)]
df = [single_df.sample(frac=1).reset_index(drop=True) for single_df in df]

for n in range(200):
    df[n]['score'] = df[n]['score'].apply(posneg)
    df[n]['parent_score'] = df[n]['parent_score'].apply(posneg)
    
    df[n]['fuzz_qr'] = df[n].apply(lambda x: fuzz.QRatio(str(x['text']), str(x['parent_text'])), axis=1)
    df[n]['fuzz_wr'] = df[n].apply(lambda x: fuzz.WRatio(str(x['text']), str(x['parent_text'])), axis=1)
    df[n]['fuzz_partial'] = df[n].apply(lambda x: fuzz.partial_ratio(str(x['text']), str(x['parent_text'])), axis=1)
    df[n]['fuzz_partial_tksort'] = df[n].apply(lambda x: fuzz.partial_token_sort_ratio(str(x['text']), str(x['parent_text'])), axis=1)
    df[n]['fuzz_tksort'] = df[n].apply(lambda x: fuzz.token_sort_ratio(str(x['text']), str(x['parent_text'])), axis=1)
    df[n]['fuzz_tkset'] = df[n].apply(lambda x: fuzz.token_set_ratio(str(x['text']), str(x['parent_text'])), axis=1)


# Now we have scrambeled dataframes of 20000 entries
df[0].head()


KeyboardInterrupt: 

In [ ]:
# text_data = []
# text_score = []

# X_train = []
# X_test = []
# y_train = []
# y_test = []

# for n in range(200):
#     text_data.append(df[n]['text'].astype(str))
#     text_score.append(df[n]['score'].apply(posneg))
    
#     X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp = train_test_split(text_data[n], text_score[n], test_size=0.20, random_state=42)
#     X_train.append(X_train_tmp)
#     X_test.append(X_test_tmp)
#     y_train.append(y_train_tmp)
#     y_test.append(y_test_tmp)

## Finally we can do some neural networks!

In [ ]:
vocab_size = len(tokenizer.word_index) + 1


model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(24, activation=tf.nn.relu))
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit(train_data,
                    y_train[current_df],
                    epochs=40,
                    batch_size=512,
                    validation_data=(test_data, y_test[current_df]),
                    verbose=1)